# Stock Candlestick Chart with Bokeh

<div class="admonition note">
  <p class="admonition-title">About the code for plots and tables</p>
  <p>
      The code for the plots and some tables is hidden on purpose.
      This is to avoid repeating code. Go to the repo to find the code.
  </p>

In [69]:
from datetime import datetime

import numpy as np
import pandas as pd
import yfinance as yf
from bokeh.models import (AdaptiveTicker, BoxAnnotation, ColumnDataSource,
                          CustomJS, CrosshairTool, DatetimeTickFormatter,
                          HoverTool, NumeralTickFormatter)
from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

## Pick stock for testing
- [GOOGL](https://finance.yahoo.com/quote/GOOGL?p=GOOGL&.tsrc=fin-srch)

In [2]:
ticker = yf.Ticker('GOOGL')
ticker_info = ticker.info
print("longName:", ticker_info.get("longName"))
print("sector:  ", ticker_info.get("sector"))
print("symbol:  ", ticker_info.get("symbol"))
print("exchange:", ticker_info.get("exchange"))

longName: Alphabet Inc.
sector:   Communication Services
symbol:   GOOGL
exchange: NMS


### Dowload prices
Download daily data for 1 year

In [3]:
ticker_hist_raw = ticker.history(period="5y")
ticker_hist_raw.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-10-03 00:00:00-04:00,60.599998,60.709000,60.107498,60.576500,26246000,0.0,0.0
2018-10-04 00:00:00-04:00,60.251499,60.294998,58.192501,58.853500,46576000,0.0,0.0
2018-10-05 00:00:00-04:00,58.799999,59.099998,57.716000,58.391499,31852000,0.0,0.0
2018-10-08 00:00:00-04:00,58.000000,58.792999,56.770000,57.796001,46190000,0.0,0.0
2018-10-09 00:00:00-04:00,57.565498,58.077499,57.208500,57.258499,33690000,0.0,0.0


## Prepare data

In [28]:
COLOR_INCREASE = "#49a3a3"
COLOR_DECREASE = "#eb3c40"
TICKET = ticker_info.get("symbol")

In [29]:
ticker_hist = (
    ticker_hist_raw
        .filter(["Open", "High", "Low", "Close"])
        .reset_index()
        .rename(columns=lambda x: x.lower())
        .assign(
            # remove timezone info
            date=lambda x: x["date"].dt.tz_localize(None),
            # move candles to the center of the day
            center=lambda x: x["date"]+pd.Timedelta('12H'),
            # define the with of candles
            width=pd.Timedelta('16H'),
            # red for price decrease, green for price increase
            color=lambda x: np.where(x["close"]> x["open"], COLOR_INCREASE, COLOR_DECREASE),
            var=lambda x: x['close'].diff(periods=1).fillna(0),
            pct_var=lambda x: (x['var']/x['open'].shift(periods=1)).fillna(0)
        )
)
ticker_hist.head()

,date,open,high,low,close,center,width,color,var,pct_var
0,2018-10-03,60.599998,60.709000,60.107498,60.576500,2018-10-03 12:00:00,0 days 16:00:00,#eb3c40,0.000000,0.000000
1,2018-10-04,60.251499,60.294998,58.192501,58.853500,2018-10-04 12:00:00,0 days 16:00:00,#eb3c40,-1.723000,-0.028432
2,2018-10-05,58.799999,59.099998,57.716000,58.391499,2018-10-05 12:00:00,0 days 16:00:00,#eb3c40,-0.462002,-0.007668
3,2018-10-08,58.000000,58.792999,56.770000,57.796001,2018-10-08 12:00:00,0 days 16:00:00,#eb3c40,-0.595497,-0.010128
4,2018-10-09,57.565498,58.077499,57.208500,57.258499,2018-10-09 12:00:00,0 days 16:00:00,#eb3c40,-0.537502,-0.009267


### define a initial range to avoid showing all the data initially

In [9]:
initial_x_max = ticker_hist["date"].max()+pd.Timedelta('1D')
initial_x_min = initial_x_max-pd.Timedelta("8W")

# this is required for when removing non-working hours
initial_x_max_int = ticker_hist.shape[0]
initial_x_min_int = initial_x_max_int-(42)

print(initial_x_min, initial_x_max)

2023-08-08 00:00:00 2023-10-03 00:00:00


In [10]:
_initial_timeframe = ticker_hist.query("date >= @initial_x_min and date <= @initial_x_max")
initial_y_min = _initial_timeframe["low"].min()*0.99
initial_y_max = _initial_timeframe["high"].max()*1.01
print(initial_y_min, initial_y_max)

125.11619728088378 140.55160369873047


## Build the candlestick plot

### other solutions for diferent colors bars

using bokeh CDSView with a filter
```
source = ColumnDataSource(ticker_hist)
inc_view = CDSView(filter=BooleanFilter((source.data["close"] > source.data["open"]).tolist()))
dec_view = CDSView(filter=BooleanFilter((source.data["close"] < source.data["open"]).tolist()))

fig.vbar(
    x="center", width="width", bottom="open", top="close", color="#eb3c40",
    source=source, view=inv_view
)

fig.vbar(
    x="center", width="width", bottom="open", top="close", color="#49a3a3",
    source=source, view=dec_view
)
```

filter directly in pandas, harder to use with ColumnDataSource
```
inc = ticker_hist["close"] > ticker_hist["open"]
dec = ticker_hist["open"] > ticker_hist["close"]
```

### basic plot

In [7]:
source = ColumnDataSource(ticker_hist)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

fig = figure(
    title="GOOGL Candlestick",
    x_axis_type="datetime",
    y_axis_label="Price $",
    tools=TOOLS,
    toolbar_location="above",
    sizing_mode="stretch_width",
    height=400,
    x_range=(initial_x_min, initial_x_max),
    y_range=(initial_y_min, initial_y_max)
)

# plot bands non working days
non_working_days = ticker_hist[['date']].assign(diff=ticker_hist['date'].diff()-pd.Timedelta('1D'))
non_working_days = non_working_days[non_working_days['diff']>=pd.Timedelta('1D')]
boxes = [
    BoxAnnotation(fill_color="#bbbbbb", fill_alpha=0.2, left=date-diff, right=date)
    for date, diff in non_working_days.values
]
fig.renderers.extend(boxes)

# plot candle wicks
fig.segment(
    x0="center", x1="center", y0="low", y1="high", color="black", source=source
)

# plot candle bars
fig.vbar(
    x="center", width="width", bottom="open", top="close", color="color",
    source=source
)

fig.add_tools(
    CrosshairTool(dimensions='both', line_color='#808080', line_alpha=0.5)
)

# date labels to x axis
fig.xaxis.formatter=DatetimeTickFormatter(
    days = '%Y-%m-%d', months = '%Y-%m', years = '%Y'
)

show(fig)

### plot without non-working days

In [8]:
source = ColumnDataSource(ticker_hist)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

fig = figure(
    title="GOOGL Candlestick without missing dates",
    x_axis_type='datetime',
    y_axis_label="Price $",
    tools=TOOLS,
    toolbar_location="above",
    sizing_mode="stretch_width",
    height=400,
    x_range=(initial_x_min_int, initial_x_max_int),
    y_range=(initial_y_min, initial_y_max),
)

# plot candle wicks
fig.segment(
    x0="index", x1="index", y0="low", y1="high", color="black", source=source
)

# plot candle bars
fig.vbar(
    x="index", width=0.6, bottom="open", top="close", color="color",
    source=source
)

fig.x_range.min_interval = 1

# one tick per week (5 weekdays)
fig.xaxis.ticker = AdaptiveTicker(
    max_interval=75, min_interval=5, num_minor_ticks=5
)

# date labels to x axis
fig.xaxis.major_label_overrides = {
    i: pd.Timestamp(d).strftime("%Y-%m-%d")
    for i, d in zip(source.data["index"], source.data["date"])
}

fig.xaxis.bounds = (source.data["index"].min(), source.data["index"].max())

show(fig)

### plot with auto y strech

In [9]:
source = ColumnDataSource(ticker_hist)

TOOLS = "xpan,wheel_zoom,box_zoom,reset,save"

fig = figure(
    title="GOOGL Candlestick with Y axis auto scaling",
    y_axis_label="Price $",
    tools=TOOLS,
    toolbar_location="above",
    sizing_mode="stretch_width",
    height=400,
    x_range=(initial_x_min_int, initial_x_max_int),
    y_range=(initial_y_min, initial_y_max),
)

# plot candle wicks
fig.segment(
    x0="index", x1="index", y0="low", y1="high", color="black", source=source
)

# plot candle bars
fig.vbar(
    x="index", width=0.6, bottom="open", top="close", color="color", source=source
)

fig.x_range.min_interval = 1

# one tick per week (5 weekdays)
fig.xaxis.ticker = AdaptiveTicker(
    max_interval=75, min_interval=5, num_minor_ticks=5
)

# date labels to x axis
fig.xaxis.major_label_overrides = {
    i: pd.Timestamp(d).strftime("%Y-%m-%d")
    for i, d in zip(source.data["index"], source.data["date"])
}

fig.xaxis.bounds = (source.data["index"].min(), source.data["index"].max())


# callback to automatically zoom the Y axis to
zoom_callback = CustomJS(
    args={'y_range': fig.y_range, 'source': source}, 
    code="""
        clearTimeout(window._autoscale_timeout);

        var Index = source.data.index,
            Low = source.data.low,
            High = source.data.high,
            start = cb_obj.start,
            end = cb_obj.end,
            min = Infinity,
            max = -Infinity;

        for (var i=0; i < Index.length; ++i) {
            if (start <= Index[i] && Index[i] <= end) {
                max = Math.max(High[i], max);
                min = Math.min(Low[i], min);
            }
        }

        var pad = (max - min) * .1;

        window._autoscale_timeout = setTimeout(function() {
            y_range.start = min - pad;
            y_range.end = max + pad;
        });
    """
)

fig.x_range.js_on_change("start", zoom_callback)

show(fig)

### add custom tooltip when hover

In [68]:
source = ColumnDataSource(ticker_hist)

TOOLS = "xpan,wheel_zoom,box_zoom,reset,save"

# generate unique key for making sure
fig_uid = "tooltip-candle" + datetime.now().strftime("%Y%m%d%H%M%S")

fig = figure(
    title="GOOGL Candlestick with tooltip",
    y_axis_label="Price $",
    tools=TOOLS,
    toolbar_location="above",
    sizing_mode="stretch_width",
    height=400,
    x_range=(initial_x_min_int, initial_x_max_int),
    y_range=(initial_y_min, initial_y_max),
    css_classes=[fig_uid] # in the case of multiple plots
)

# plot candle wicks
fig.segment(
    x0="index", x1="index", y0="low", y1="high", color="black", source=source
)

# plot candle bars
candles = fig.vbar(
    x="index", width=0.6, bottom="open", top="close", color="color",
    source=source
)

fig.add_tools(CrosshairTool(dimensions='both', line_color='#808080', line_alpha=0.5))

# max zoom
fig.x_range.min_interval = 1

# one tick per week (5 weekdays)
fig.xaxis.ticker = AdaptiveTicker(max_interval=75, min_interval=5, num_minor_ticks=5)

# date labels to x axis
fig.xaxis.major_label_overrides = {
    i: pd.Timestamp(d).strftime("%Y-%m-%d")
    for i, d in zip(source.data["index"], source.data["date"])
}

fig.xaxis.bounds = (source.data["index"].min(), source.data["index"].max())

# callback to automatically zoom the Y axis to
zoom_callback = CustomJS(
    args={'y_range': fig.y_range, 'source': source}, 
    code="""
        clearTimeout(window._autoscale_timeout);
        var Index = source.data.index,
            Low = source.data.low,
            High = source.data.high,
            start = cb_obj.start,
            end = cb_obj.end,
            min = Infinity,
            max = -Infinity;
        for (var i=0; i < Index.length; ++i) {
            if (start <= Index[i] && Index[i] <= end) {
                max = Math.max(High[i], max);
                min = Math.min(Low[i], min);
            }
        }
        var pad = (max - min) * .1;
        window._autoscale_timeout = setTimeout(function() {
            y_range.start = min - pad;
            y_range.end = max + pad;
        });
    """
)

fig.x_range.js_on_change("start", zoom_callback)

numberformat = "0,0.00"
xaxis_dt_format = '%Y-%m-%d'
pctformat = '0.00%'

# format y axis
fig.yaxis[0].formatter = NumeralTickFormatter(format=numberformat)
# COLOR_INCREASE, COLOR_DECREASE
tooltip_callback = CustomJS(
    args={"source": source}, 
    code=f"""
    var tooltips = document.getElementsByClassName("bk-Figure {fig_uid}")[0]
        .shadowRoot
        .querySelector(".bk-Canvas")
        .shadowRoot
        .querySelector(".bk-Tooltip")
    
    if (tooltips !== null) {{
        tooltips.style.top = "32px";
        tooltips.style.left = "75px";
        tooltips.style.width = "500px";
        tooltips.style.border = "none"
        
        
        var curr_indice = cb_data.index.indices;
        var curr_open = source.data.open[curr_indice];
        var curr_close = source.data.close[curr_indice];
        
        var _color = "{COLOR_DECREASE}"
        if (curr_open <= curr_close) {{
            _color = "{COLOR_INCREASE}"
        }}
        
        var tipspan = tooltips
            .shadowRoot
            .querySelector(".bk-tooltip-content")
            .querySelectorAll(".tooltip-span-color")
        for (var i = 0, len = tipspan.length; i < len; i ++) {{
            tipspan[i].style.color = _color;
        }}

    }}
    
    // var tooltips = document.getElementsByClassName("bk-Tooltip");
    // for (var i = 0, len = tooltips.length; i < len; i ++) {{
    //    tooltips[i].style.top = "10px";
    //    tooltips[i].style.left = "50px";
    //    tooltips[i].style.width = "500px";
    //    tooltips[i].style.border = "none"
    //}}
    """
)

tooltips = f"""
    <span style="font-weight: bold; margin-right: 5px">{TICKET}</span>
    <span>O</span>
    <span class="tooltip-span-color" style="margin-right: 5px">
        @open{{{numberformat}}}
    </span>
    <span>H</span>
    <span class="tooltip-span-color" style="margin-right: 5px">
        @high{{{numberformat}}}
    </span>
    <span>L</span>
    <span class="tooltip-span-color" style="margin-right: 5px">
        @low{{{numberformat}}}
    </span>
    <span>C</span>
    <span class="tooltip-span-color" style="margin-right: 5px">
        @close{{{numberformat}}}
    </span>
    <span class="tooltip-span-color" style="margin-right: 10px">
        @var{{{numberformat}}} (@pct_var{{{pctformat}}})
    </span>
"""

_price_tooltip = lambda x: f"$ @{x}{{"+numberformat+"}"
fig.add_tools(HoverTool(
    renderers=[candles],
    tooltips=tooltips,
    # formatters={'@date': 'datetime'},
    mode='vline',
    toggleable=False,
    show_arrow=False,
    line_policy="none",
    callback=tooltip_callback,
))

show(fig)

# References
- https://github.com/ndepaola/bokeh-candlestick/blob/master/candlestick_plot.py